In [ ]:
print('Setup complete.')

# Lab 04: Planning Agents (Plan-and-Execute)

## Learning Objectives
- Understand the Plan-and-Execute agent architecture
- Differentiate between a 'Planner' and an 'Executor'
- Implement a system where one LLM creates a plan and another executes it
- See how structured planning can solve more complex, multi-step goals

## Setup

In [ ]:
import json
from typing import List, Dict, Any

## Part 1: The Planner and Executor Roles

The Plan-and-Execute model splits the problem-solving process into two distinct phases:

1.  **Planner**: An LLM that receives the user's high-level goal and breaks it down into a sequence of concrete, executable steps. It does not use tools; it only thinks and plans.
2.  **Executor**: An agent (often a ReAct-style agent) that takes one step from the plan at a time and executes it. The executor has access to tools and is focused on completing its single task.

This separation of concerns makes the agent more robust. The planner can focus on the high-level strategy without getting bogged down in execution details, and the executor can focus on the immediate step without needing to understand the overall goal.

In [ ]:
# --- Mock Tools (reused from previous labs) ---
def search(query: str) -> str:
    if 'french revolution' in query.lower():
        return 'The French Revolution was a period of social and political upheaval in France from 1789 to 1799.'
    if 'napoleon bonaparte' in query.lower():
        return 'Napoleon Bonaparte was a French military and political leader who rose to prominence during the French Revolution.'
    return 'Information not found.'

def write_file(filename: str, content: str) -> str:
    print(f'--- Writing to file: {filename} ---
{content}
--------------------')
    return f'Successfully wrote content to {filename}.'

# --- Mock LLMs for Planner and Executor ---
class PlannerLLM:
    def create_plan(self, goal: str) -> Dict[str, List[Dict[str, str]]]:
        if 'french revolution' in goal.lower() and 'napoleon' in goal.lower():
            plan = {
                'steps': [
                    {'step': 1, 'instruction': 'Find information about the French Revolution.', 'tool': 'search', 'args': 'French Revolution'},
                    {'step': 2, 'instruction': 'Find information about Napoleon Bonaparte.', 'tool': 'search', 'args': 'Napoleon Bonaparte'},
                    {'step': 3, 'instruction': 'Combine the gathered information into a summary.', 'tool': 'none', 'args': ''},
                    {'step': 4, 'instruction': 'Write the summary to a file named summary.txt.', 'tool': 'write_file', 'args': {'filename': 'summary.txt', 'content': '<summary_from_step_3>'}}
                ]
            }
            return plan
        return {'steps': []}

class ExecutorAgent:
    def __init__(self, tools: Dict):
        self.tools = tools

    def execute_step(self, step: Dict, context: Dict) -> str:
        tool_name = step['tool']
        if tool_name == 'none':
            # This step is for internal processing, like summarizing
            info1 = context.get('step_1_result', '')
            info2 = context.get('step_2_result', '')
            return f'Summary: {info1} {info2}'
        
        tool_function = self.tools.get(tool_name)
        if not tool_function:
            return f'Error: Tool {tool_name} not found.'
            
        # Handle argument substitution from context
        args = step['args']
        if isinstance(args, dict):
            if args['content'] == '<summary_from_step_3>':
                args['content'] = context.get('step_3_result', '')
            return tool_function(**args)
        else:
            return tool_function(args)


## Part 2: The Plan-and-Execute Loop

In [ ]:
def run_plan_and_execute(goal: str):
    # 1. Planning Phase
    print(f'Goal: {goal}')
    planner = PlannerLLM()
    plan = planner.create_plan(goal)
    print(f'\nPlanner created a {len(plan["steps"])}-step plan.')
    for s in plan['steps']:
        print(f'  - Step {s["step"]}: {s["instruction"]}')
    
    # 2. Execution Phase
    print("
--- Starting Execution ---")
    available_tools = {'search': search, 'write_file': write_file}
    executor = ExecutorAgent(tools=available_tools)
    execution_context = {} # Stores results from previous steps
    
    for step in plan['steps']:
        print(f'\nExecuting Step {step["step"]}: {step["instruction"]}')
        result = executor.execute_step(step, execution_context)
        print(f'Result: {result}')
        
        # Store the result in the context for future steps
        execution_context[f'step_{step["step"]}_result'] = result
        
    print("
--- Plan Finished ---")
    return execution_context.get(f'step_{len(plan["steps"])}_result')

# --- Run the Agent ---
goal = 'Research the French Revolution and Napoleon, then write a summary to a file.'
final_result = run_plan_and_execute(goal)

## Exercises

1. **Implement Plan Re-evaluation**: What if a step fails? In the `run_plan_and_execute` loop, if an executor step returns an error, the agent should stop, send the original goal and the error back to the `PlannerLLM`, and ask it to generate a *new* plan. This creates a feedback loop.
2. **Dynamic Argument Substitution**: The current argument substitution is hard-coded (`<summary_from_step_3>`). Generalize it. The planner should be able to specify a placeholder like `{{step_2_result}}` in the arguments, and the executor should be able to replace it with the actual value from the `execution_context`.
3. **Create a More Complex Plan**: Think of a more complex goal, like "Find the top 3 trending news articles, summarize each one, and save them to three separate files." Modify the `PlannerLLM` to generate a plan for this goal. You will likely need to add a new tool, `get_trending_articles()`.

## Summary

You learned:
- The **Plan-and-Execute** architecture, which separates high-level planning from low-level execution.
- The role of the **Planner** in decomposing a complex goal into a structured, step-by-step plan.
- The role of the **Executor** in carrying out each step of the plan using a set of available tools.
- How this architecture enables agents to solve more complex problems by maintaining a clear strategy while handling execution details separately.